In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2024-05-05 18:57:55.024340: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 18:57:55.024488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 18:57:55.172404: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_df = pd.read_csv("/kaggle/input/smtitanic/train.csv")
test_df = pd.read_csv("/kaggle/input/smtitanic/test.csv")

# Dropping null values from the dataset.
train_df.dropna(subset = ["Age"], inplace = True)
test_df.dropna(subset = ["Age"], inplace = True)

# All out training data
train_x_age = train_df["Age"]
train_x_gender = train_df["Sex"].map({ "male": 1, "female": 2})
train_x_class = train_df["Pclass"]

train_y = train_df["Survived"]

In [3]:
def build_model(lr):
    
    # This is were the model is declared and all the layers.
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(6, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr),
                      loss="mean_squared_error",
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

def train_model(model, train_x, train_y, ep, bs, vs):
    model.fit(train_x, train_y, epochs = ep, batch_size = bs, validation_split = vs)

In [4]:
ep = 4
bs = 8
lr = 0.01
vs = 0.6

model = build_model(lr)
train_model(model, [train_x_age, train_x_gender, train_x_class], train_y, ep, bs, vs)

Epoch 1/4
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.3102 - root_mean_squared_error: 0.5562 - val_loss: 0.2566 - val_root_mean_squared_error: 0.5066
Epoch 2/4
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2758 - root_mean_squared_error: 0.5251 - val_loss: 0.2476 - val_root_mean_squared_error: 0.4976
Epoch 3/4
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2664 - root_mean_squared_error: 0.5160 - val_loss: 0.3245 - val_root_mean_squared_error: 0.5697
Epoch 4/4
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2737 - root_mean_squared_error: 0.5227 - val_loss: 0.3249 - val_root_mean_squared_error: 0.5700


In [5]:
test_df_gender = test_df["Sex"].map({ "male": 1, "female": 2})
test_df_age = test_df["Age"]
test_df_class = test_df["Pclass"]

test_df_survived = np.reshape(model.predict([test_df_age, test_df_gender]), (332,)) * 100

# We create a data frame to compare our predictions with the input
test_results = pd.DataFrame()
test_results["Gender"] = test_df_gender.map({ 1: "male", 2: "female"})
test_results["Age"] = test_df_age
test_results["Pclass"] = test_df_class
test_results["Survival Prediction"] = test_df_survived

test_results.head(10)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


,Gender,Age,Pclass,Survival Prediction
0,male,34.5,3,7.449392
1,female,47.0,3,3.061157
2,male,62.0,2,1.017006
3,male,27.0,3,12.365915
4,female,22.0,3,17.023512
5,male,14.0,3,27.187332
6,female,30.0,3,10.130705
7,male,26.0,2,13.200185
8,female,18.0,3,21.677652
9,male,21.0,3,18.107084
